In [7]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
import time
from random import randint

In [8]:
options = webdriver.ChromeOptions()
options.add_argument("download.default_directory=" + os.getcwd() + "/Downloads")

In [9]:
def google_scholar_scraper(record,driver,j):
    
    # Set driver to go to start_page and search a given name with valid scholar profile    
    google_scholar = "https://scholar.google.com/citations?hl=en&view_op=search_authors&mauthors=&btnG="
    driver.get(google_scholar)

    time.sleep(randint(4,7))
    
    # Whether a profile is found is set to false by default
    profile_found = False
    search_bar = driver.find_element_by_class_name('gs_in_txt')
    search_bar.clear()
    search_bar.send_keys("\"" + record.faculty.strip() + "\"" + " CSUN")
    search_bar.send_keys(Keys.ENTER)
    
    # If there's search results
    if not driver.find_elements_by_xpath('//*[@id="gsc_sa_ccl"]/div/p[1]'):
        # A profile was found
        profile_found = True
    # If there's no search results let's try again
    else:
        # A little wait
        time.sleep(randint(4,7))
        search_bar = driver.find_element_by_class_name('gs_in_txt')
        search_bar.clear()
        search_bar.send_keys("\"" + record.faculty.strip() + "\"" + " Northridge")
        search_bar.send_keys(Keys.ENTER)
        if not driver.find_elements_by_xpath('//*[@id="gsc_sa_ccl"]/div/p[1]'):
            profile_found = True
            
    # If the profile was found
    if profile_found:
        time.sleep(randint(4,7))
        view_profile_link = driver.find_element_by_xpath('//*[@id="gsc_sa_ccl"]/div[1]/div/div/h3/a').get_attribute("href")
        citation_collector(record, view_profile_link,j)
    else:
        f = open("google_scholar_no_results.csv", "a")
        f.write(record.faculty.strip() + ",")
        f.write(record.college + ",")
        f.write(record.department + ",")
        f.write(record.url + "\n")
        f.close()
        
        

In [ ]:
def citation_collector(record, url,j):
    
    driver = webdriver.Chrome()
    driver.implicitly_wait(15)
    driver.get(url)
    
    # To get all the links we need to click the Show More multiple times to show the whole links 
    gs_bpf_wrapper = driver.find_element_by_id("gsc_bpf_more")

    while not gs_bpf_wrapper.get_attribute("disabled"):
        gs_bpf_wrapper.click()
        time.sleep(3)

    #pull the first citation for the page
    view_citation_links = driver.find_elements_by_class_name('gsc_a_at')

    citation_file = open('google_scholar_citations.csv', "a")
    
    num_links = len(view_citation_links)

    for i in range(j,len(view_citation_links)):

        not_his_paper = False

        citation = ""
        title = view_citation_links[i].text
        paper_type = ""
        book = ""
        journal = ""
        date = ""
        authors = ""
        publisher = ""
        description = ""
        edition = ""
        series = ""
        volume = ""
        issue = ""
        chapter = ""
        pages = ""
        issn = ""
        isbn = ""
        handle = ""


        time.sleep(randint(4,6))
        
        view_citation_links[i].click()
        if driver.find_elements_by_xpath('//*[@id="gsc_vcd_title"]/a'):
            url = driver.find_element_by_xpath('//*[@id="gsc_vcd_title"]/a').get_attribute("href")
        else:
            url = ""
        
        time.sleep(randint(4,6))

        citation_table = driver.find_element_by_id("gsc_vcd_table")
        citation_table_data_objects= citation_table.find_elements_by_class_name("gs_scl")

        data_category = citation_table_data_objects[0].find_element_by_tag_name("div")

        if data_category.text == "Authors":

            i = 0
            for data in citation_table_data_objects:

                data_category = citation_table_data_objects[i].find_element_by_tag_name("div")

                citation_title = driver.find_element_by_id('gsc_vcd_title')
                title = citation_title.text

                if data_category.text == "Authors":
                    data_field = data.find_element_by_class_name("gsc_vcd_value")
                    if "|" in data_field.text:
                        authors = data_field.text
                        authors.replace("|", " ")
                    else:
                        authors = data_field.text
                    # what second condition was i going to check here?
                    if not record.faculty.strip() in data_field.text:
                        not_his_paper = True
                elif data_category.text == "Publication date":
                    data_field = data.find_element_by_class_name("gsc_vcd_value")
                    date = data_field.text
                elif data_category.text == "Publisher":
                    data_field = data.find_element_by_class_name("gsc_vcd_value")
                    publisher = data_field.text
                elif data_category.text == "Description":
                    # this line of code grabs the description
                    data_field = data.find_element_by_class_name("gsc_vcd_value")
                    data = data_field.text

                    # here is where we want to go through the description and
                    # replace every instance of "\n" with " "
                    desc = data.split("\n")
                    description = ""
                    for piece in desc:
                        description += piece + " "
                elif data_category.text == "Journal":
                    data_field = data.find_element_by_class_name("gsc_vcd_value")
                    if "|" in data_field.text:
                        journal = data_field.text
                        journal.replace("|", " ")
                    else:
                        journal = data_field.text
                    journal = data_field.text
                elif data_category.text == "Issue":
                    data_field = data.find_element_by_class_name("gsc_vcd_value")
                    issue = data_field.text
                elif data_category.text == "Volume":
                    data_field = data.find_element_by_class_name("gsc_vcd_value")
                    volume = data_field.text
                elif data_category.text == "Pages":
                    data_field = data.find_element_by_class_name("gsc_vcd_value")
                    pages = data_field.text
                i = i + 1

            citation_file.write(record.faculty + "|" + citation + "|" + title + "|" + paper_type + "|")
            citation_file.write(description + "|" + book + "|" + journal + "|" + date + "|")
            citation_file.write(authors + "|" + publisher + "|" + edition + "|" + series + "|")
            citation_file.write(volume + "|" + issue + "|" + chapter + "|" + pages + "|")
            citation_file.write(issn + "|" + isbn + "|" + handle + "|" + url + "\n")

        elif data_category.text == "Inventors":
            pass

        elif not_his_paper:
            print("Error: Publication ", i , " did not get captured by program; new decision tree required.")

        close_table = driver.find_element_by_id("gs_md_cita-d-x")     
        close_table.click()
        time.sleep(randint(4,6))

    citation_file.close()
    driver.close()
    

In [19]:
view_content = pd.read_csv("catalog_content.csv")
view_content.head()
print(view_content.columns)

Index(['faculty', 'college', 'department', 'url'], dtype='object')


In [20]:
view_content.iloc[235] #Self trapping of 

faculty                                         Anna Bezryadina
college                                        Science and Math
department                    Department of Physics & Astronomy
url            https://www.csun.edu/science-mathematics/phys...
Name: 235, dtype: object

In [22]:
try:
    driver = webdriver.Chrome()
    driver.implicitly_wait(15)
    google_scholar_scraper(view_content.iloc[235],driver,18)
finally:
    driver.close()

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=46823): Max retries exceeded with url: /session/3c17ccaca76ff803ffa0d989505199f3/window (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f91924f8cc0>: Failed to establish a new connection: [Errno 111] Connection refused'))